### EX 5.9 Stuff_LCEL_RAG

LCEL(LangChain Expression Language)를 사용하여 Stuff RAG를 구현해보자.
Stuff RAG는 문서 조각을 탐색하여 유저가 원하는 내용이 있는 문서 조각을 발견할 때마다 prompt에 기록한다. 그리고 모은 문서조각을 확인해서 유저에게 문서의 내용에 답변하는 알고리즘이다.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is described as having glass doors, a hallway that smells of boiled cabbage and old rag mats, and being seven flights up. The building has a faulty lift, and the flat where Winston lives is on the seventh floor. Inside the flat, there is a telescreen that cannot be completely shut off, and there is a poster with an enormous face that reads "BIG BROTHER IS WATCHING YOU."')